## Load all datasets

In [ ]:
import os,json
from datasets import load_dataset

pair_sets = [ "Cardillo", "Jankowiak"]
quad_sets = [ "SAT*", ]#'Green']

all_data ={
    "Cardillo": load_dataset('json', data_files='../../datasets/Cardillo/cardillo.jsonl'),
    "Jankowiak":load_dataset('json', data_files='../../datasets/Jankowiak/Jankowiak.jsonl'),
    "SAT*":load_dataset('json', data_files='../../datasets/SAT_STAR/sat_star.jsonl'),
    "SAT":load_dataset('json', data_files='../../datasets/sat/sat.jsonl')
    #Green
}



## Create all datasets utils dictionaries
- datasets_sent2label
- datasets_sent2idx
- datasets_idx2sent

In [ ]:
datasets_sent2label = {}
datasets_sent2idx = {}
datasets_idx2sent = {}


for dataset in pairs_sets:
    datasets_sent2label[dataset] = {}
    datasets_sent2idx[dataset] = {}
    datasets_idx2sent[dataset] = {}
    n = 0
    for i,e in enumerate(all_data[dataset]["test"]):
        for j,f in enumerate(e["sentences"]):
            datasets_sent2label[dataset][f] = e["labels"][j]
            datasets_sent2idx[dataset][f] = n
            datasets_idx2sent[dataset][n] = f
            n+=1
            
            
for dataset in quad_sets:
    datasets_sent2label[dataset] = {}
    datasets_sent2idx[dataset] = {}
    datasets_idx2sent[dataset] = {}
    n = 0
    for i,e in enumerate(all_data[dataset]["test"]):
        for j,f in enumerate(e["pairs"]):
            t = eval(e["stem"])
            t.extend(f)
            t = tuple(t)
            datasets_sent2label[dataset][t] = e["labels"][j]
            datasets_sent2idx[dataset][t] = n
            datasets_idx2sent[dataset][n] = t
            n+=1

## All models perplexities

In [ ]:
ppl_scores = "asahi_results/metaphor_results/scores_no_prompt/"
ppl_openai_scores = "asahi_results/metaphor_results/scores_openai_no_prompt/"   

no_prompt = os.listdir(ppl_scores)
no_prompt_openai = os.listdir(ppl_openai_scores)


In [ ]:
results_no_prompt = {}

for x in no_prompt+no_prompt_openai:
    p = json.load(open(asahi_no_prompt+"/"+x))
    x = x.replace(".Metaphors_and_Analogies_","|")
    w = x.split("|")
    model,dataset=w[0],w[1][:-10]
    if not dataset in results_no_prompt:
        results_no_prompt[dataset]={}
    if not model in results_no_prompt[dataset]:    
        results_no_prompt[dataset][model] = {"text":[],"true":sum(p["labels"],[]),"idx":[],"score":[]}
    for i,y in enumerate(p["perplexity"]):
        results_no_prompt[dataset][model]["score"].append(y["score"])
        results_no_prompt[dataset][model]["idx"].append(y["index"])
        results_no_prompt[dataset][model]["text"].append(datasets_idx2sent[dataset][i])
        assert results_no_prompt[dataset][model]["true"][i]==datasets_sent2label[dataset][datasets_idx2sent[dataset][i]]



In [ ]:
output = "utils_dictionaries/"
with open(output+"results_no_prompt_zero_shot.json", "w") as outfile:
    json.dump(results_no_prompt,outfile,indent="\t")

## SAT_MET Dictionaries 
- satsetid : sat quadruples to set_ids
- satlabs : quadruples to labels


In [ ]:
satlabs = {}
satsetid = {}
for x in all_data["SAT*"]:
    for i,pair in enumerate(x["pairs"]):
        target = eval(x["stem"])
        target.extend(pair)
        satlabs[tuple(target)]=x["labels"][i]
        satsetid[tuple(target)]=x["id"]


### SAT_met no prompt

In [ ]:
res_dir = "asahi_results/sat-scores-no-prompt-other-models/"
a = os.listdir(res_dir)
a = [x for x in a if "sat_full_None.prompt" in x]
didisat = {}

sat_no_score = {}
for x in a:
    p = json.load(open(res_dir+x))
    model = (x[:-26])
    sat_no_score[model]={"text":[],"score":[],"true":[],"idx":[]}
    for i,y in enumerate(p):
        q = {}
        text = y["input"]+y["output"]
        flag = 0
        for quad in satlabs:
            c = 0
            for word in quad:
                if word in text :
                    c+=1
            if c==4:
                mes4mots = quad
                flag = 1
        if flag==1:
            sat_no_score[model]["score"].append(y["score"])
            sat_no_score[model]["text"].append(mes4mots)
            sat_no_score[model]["true"].append(satlabs[mes4mots])
            sat_no_score[model]["idx"].append(satsetid [mes4mots])
            didisat[i]=mes4mots


In [ ]:
p = json.load(open("asahi_results/gpt3-scores-noprompt-SAT/davinci_sat_full_None.prompt.json"))
sat_no_score["davinci"] = {"text":sat_no_score[model]["text"],"score":[],"true":sat_no_score[model]["true"],"idx":sat_no_score[model]["idx"]}
n = 0
for x in p:
    for y in x[1]:
        if n in didisat:
            sat_no_score["davinci"]["score"].append(y)
        n+=1

In [ ]:
output = "utils_dictionaries/"
with open(output+"results_sat_no_prompt_zero_shot.json", "w") as outfile:
    json.dump(sat_no_score,outfile,indent="\t")